In [1]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

In [2]:
genre_data = pd.read_csv('artists.csv')


In [3]:
genre_data.head()


,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0


In [4]:
genre_data.dtypes

id             object
followers     float64
genres         object
name           object
popularity      int64
dtype: object

In [5]:
genre_data['genres'].values[0]

'[]'

In [6]:
genre_data['genres'].values[0][0]

'['

In [7]:
genre_data['genres_upd'] = genre_data['genres'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", x)])

In [8]:
genre_data['genres_upd'].values[0]


[]

In [9]:
spotify_df = pd.read_csv('tracks.csv')
spotify_df['artists_upd_v1'] = spotify_df['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))


In [10]:
spotify_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,...,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3,[Uli]
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,...,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1,[Fernando Pessoa]
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,...,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5,[Ignacio Corsini]
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,...,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3,[Ignacio Corsini]
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,...,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4,[Dick Haymes]


In [11]:
spotify_df['artists_upd_v1'].values[0]

['Uli']

In [12]:
spotify_df['artists_upd_v1'].values[0][0]

'Uli'

In [13]:
spotify_df[spotify_df['name']=='Adore You']

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1
86217,5AnCLGg35ziFOloEnXK4uu,Adore You,71,278747,0,['Miley Cyrus'],['5YGY8feqx7naU7z4HrwZM6'],2013-10-04,0.583,0.655,...,-5.407,1,0.0315,0.1110,0.000004,0.113,0.201,119.759,4,[Miley Cyrus]
91884,3jjujdWJ72nww5eGnfs2E7,Adore You,88,207133,0,['Harry Styles'],['6KImCVD70vtIoJWnq6nGn3'],2019-12-13,0.676,0.771,...,-3.675,1,0.0483,0.0237,0.000007,0.102,0.569,99.048,4,[Harry Styles]
92524,1M4qEo4HE3PRaCOM7EXNJq,Adore You,74,207133,0,['Harry Styles'],['6KImCVD70vtIoJWnq6nGn3'],2019-12-06,0.676,0.771,...,-3.675,1,0.0483,0.0237,0.000007,0.102,0.569,99.048,4,[Harry Styles]


In [14]:
spotify_df['artists_upd_v2'] = spotify_df['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
spotify_df['artists_upd'] = np.where(spotify_df['artists_upd_v1'].apply(lambda x: not x), spotify_df['artists_upd_v2'], spotify_df['artists_upd_v1'] )

In [15]:
spotify_df['artists_upd'] = spotify_df['artists_upd'].fillna('Unknown')

In [16]:
spotify_df['name'] = spotify_df['name'].fillna('Unknown')

In [17]:
spotify_df['artists_upd'] = spotify_df['artists_upd'].fillna('Unknown')
spotify_df['name'] = spotify_df['name'].fillna('Unknown')

In [18]:
spotify_df = spotify_df.dropna(subset=['artists_upd', 'name'])

In [19]:
spotify_df['artists_song'] = spotify_df.apply(lambda row: (row['artists_upd'][0] if row['artists_upd'] else 'Unknown') + row['name'], axis=1)

In [20]:
print(spotify_df['artists_song'].isna().sum())

0


In [21]:
spotify_df.sort_values(['artists_song','release_date'], ascending = False, inplace = True)

In [37]:
spotify_df[spotify_df['name']=='So Good']

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists
466371,2tPEU0AUk8ha6aSl4uHVMA,So Good,37,193840,0,"[""Destiny's Child""]",['1Y8cdNmUJH7yBTd9yOvr5i'],1999-09-07,0.886,0.674,...,0.0,0.0369,0.622,97.940,4,[],[Destiny's Child],[Destiny's Child],Destiny's ChildSo Good,"[pop, hip_pop, girl_group, urban_contemporary,..."
466372,1pLJSpfeHXsIyJS1BmKrrr,So Good,52,193933,0,"[""Destiny's Child""]",['1Y8cdNmUJH7yBTd9yOvr5i'],1999-07-27,0.882,0.683,...,0.0,0.0369,0.618,97.932,4,[],[Destiny's Child],[Destiny's Child],Destiny's ChildSo Good,"[pop, hip_pop, girl_group, urban_contemporary,..."
511842,7prj8sf0M0de5f5PqnwQOV,So Good,1,149987,0,['Buddy & Ella Johnson'],['17Z8L8nhCrjHA1NvQYS5OI'],1953,0.724,0.620,...,0.0,0.0710,0.886,73.206,4,[Buddy & Ella Johnson],[],[Buddy & Ella Johnson],Buddy & Ella JohnsonSo Good,[]
545156,590ZheHFaC3JsZBwL8otpk,So Good,54,210280,0,['B.o.B'],['5ndkK3dpZLKtBklKjxNQwT'],2012-05-01,0.585,0.896,...,0.0,0.2250,0.599,170.946,4,[B.o.B],[],[B.o.B],B.o.BSo Good,"[pop, southern_hip_hop, rap, pop_rap, dance_po..."
545157,28GUjBGqZVcAV4PHSYzkj2,So Good,66,213253,0,['B.o.B'],['5ndkK3dpZLKtBklKjxNQwT'],2012-04-27,0.660,0.900,...,0.0,0.2190,0.591,85.510,4,[B.o.B],[],[B.o.B],B.o.BSo Good,"[pop, southern_hip_hop, rap, pop_rap, dance_po..."
574772,7qYB1CvkOxXqnn0MebDbU3,So Good,20,279800,0,['Al Jarreau'],['3YR92OLKlvkK5oKNekSqXe'],1988,0.558,0.303,...,0.0,0.1290,0.329,143.998,3,[Al Jarreau],[],[Al Jarreau],Al JarreauSo Good,"[smooth_jazz, soft_rock, yacht_rock, quiet_sto..."


In [23]:
artists_exploded = spotify_df[['artists_upd','id']].explode('artists_upd')

In [24]:
print(artists_exploded.columns)
print(genre_data.columns)

Index(['artists_upd', 'id'], dtype='object')
Index(['id', 'followers', 'genres', 'name', 'popularity', 'genres_upd'], dtype='object')


In [25]:
artists_exploded_enriched = artists_exploded.merge(genre_data, how='left', left_on='artists_upd', right_on='name')
artists_exploded_enriched_nonnull = artists_exploded_enriched[~artists_exploded_enriched.genres_upd.isnull()]

In [26]:
print(artists_exploded_enriched_nonnull.columns)

Index(['artists_upd', 'id_x', 'id_y', 'followers', 'genres', 'name',
       'popularity', 'genres_upd'],
      dtype='object')


In [27]:
artists_exploded_enriched_nonnull[artists_exploded_enriched_nonnull['id_x'] == '6KuQTIu1KoTTkLXKrwlLPV']

,artists_upd,id_x,id_y,followers,genres,name,popularity,genres_upd
216506,Robert Schumann,6KuQTIu1KoTTkLXKrwlLPV,2UqjDAXnDxejEyE0CzfUrZ,423826.0,"['classical', 'early romantic era', 'german ro...",Robert Schumann,64.0,"[classical, early_romantic_era, german_romanti..."
216507,Vladimir Horowitz,6KuQTIu1KoTTkLXKrwlLPV,4Ws5hSoABAwvGJ4LhHwHgq,92365.0,"['classical', 'classical performance', 'classi...",Vladimir Horowitz,54.0,"[classical, classical_performance, classical_p..."


In [28]:
artists_genres_merged = artists_exploded_enriched_nonnull.groupby('id_x')['genres_upd'].apply(list).reset_index()

In [29]:
artists_genres_merged['consolidates_genre_lists'] = artists_genres_merged['genres_upd'].apply(lambda x: list(set(list(itertools.chain.from_iterable(x)))))

In [30]:
artists_genres_merged.head()

,id_x,genres_upd,consolidates_genre_lists
0,0004Uy71ku11n3LMpuyf59,[[polish_rock]],[polish_rock]
1,000CSYu4rvd8cQ7JilfxhZ,"[[country_quebecois, rock_quebecois]]","[rock_quebecois, country_quebecois]"
2,000DsoWJKHdaUmhgcnpr8j,[[barnmusik]],[barnmusik]
3,000G1xMMuwxNHmwVsBdtj1,"[[candy_pop, new_wave, new_wave_pop, permanent...","[power_pop, new_wave, permanent_wave, rock, ca..."
4,000KblXP5csWFFFsD6smOy,"[[chamame, folclore_salteno, folklore_argentino]]","[folklore_argentino, chamame, folclore_salteno]"


In [31]:
print(spotify_df.columns)

Index(['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artists',
       'id_artists', 'release_date', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature', 'artists_upd_v1',
       'artists_upd_v2', 'artists_upd', 'artists_song'],
      dtype='object')


In [32]:
spotify_df = spotify_df.merge(artists_genres_merged[['id_x', 'consolidates_genre_lists']], left_on='id', right_on='id_x', how='left')

In [33]:
spotify_df.drop(columns=['id_x'], inplace=True)

In [34]:
spotify_df.tail()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists
586667,3YCC4oCzagcfx4s3fp3XFz,Bandolera,35,322926,1,"[""At' Fat"", 'Yemil', 'Boza', 'El Tachi']","['455JBMYIT1lWf1Djj4rDc7', '7g4cPtKxNx146qdaY9...",2020-11-17,0.502,0.814,...,0.00000,0.1510,0.6720,72.344,4,"[ Fat"", , , , , ]",[At' Fat],"[ Fat"", , , , , ]","Fat"", Bandolera",NaN
586668,5sr38tHMSPpaENiNcjUpFi,Cathrina,10,184560,0,"[""Henry D' Cruz"", ""Helen D'Souza""]","['3nH2WLSi1MhvlmJNkbLbIv', '4NR8lI9YoPWo5v2Kv6...",1976-01-01,0.661,0.677,...,0.00154,0.0361,0.8090,116.855,4,"[ Cruz"", ""Helen D]","[Henry D' Cruz, Helen D'Souza]","[ Cruz"", ""Helen D]","Cruz"", ""Helen DCathrina",NaN
586669,6Pu2leLWEfThVIqJilw6O9,The Passion of Our Lord according to St. Matth...,4,79027,0,"[""Boys' Choir of The Church of The Transfigura...","['47EwjzhfZKotjVtvlQFNFS', '5H155SiWINLkYRElqw...",1963,0.177,0.208,...,0.94600,0.0982,0.0587,132.053,3,"[ Choir of The Church of The Transfiguration"",...",[Boys' Choir of The Church of The Transfigurat...,"[ Choir of The Church of The Transfiguration"",...","Choir of The Church of The Transfiguration"", ...",NaN
586670,7f092ECp06XbFS6Ms5Yk6R,Mia Oraia Petalouda,26,100313,0,"[""Typaldos' Children's Choire""]",['7JJrBciIAQBPOBB5U6u49p'],2014-06-08,0.773,0.183,...,0.00000,0.1190,0.8640,79.950,4,[ Children],[Typaldos' Children's Choire],[ Children],ChildrenMia Oraia Petalouda,NaN
586671,1yw3fl6UGrxjq3BTeJh6xz,I Koukouvagia,28,96508,0,"[""Typaldos' Children's Choire""]",['7JJrBciIAQBPOBB5U6u49p'],2014-06-08,0.748,0.256,...,0.00000,0.1200,0.9240,179.898,4,[ Children],[Typaldos' Children's Choire],[ Children],ChildrenI Koukouvagia,NaN
